In [ ]:
!pip install pandas numpy scipy scikit-learn matplotlib seaborn cvxopt xgboost shap kaggle

!git clone https://github.com/drn-i/WESAD
!git clone https://github.com/lciti/cvxEDA.git
!cp /content/cvxEDA/src/* /content/WESAD/
!rm -rf /content/cvxEDA
!rm -rf /content/sample_data

!mv /content/WESAD/* ./
!rm -rf /content/WESAD

# Line 277
# df['label'] = df[['0', '1', '2']].idxmax(axis=1).astype(int)

# Line 209
# feat_names.append('_'.join([str(row), str(col)]))


LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///kaggle/working/In[7]#1:12\[90mIn[7]:1:12[0;0m]8;;\
;python3 -m[48;2;120;70;70m pip install pandas numpy scipy scikit-learn matplotlib seaborn cvxopt xgboost kaggle[0;0m
[90m#          └───────────────────────────────────────────────────────────────────────────────────┘ ── [0;0m[91mextra tokens after end of expression[0;0m

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("orvile/wesad-wearable-stress-affect-detection-dataset")

print("Path to dataset files:", path)

!mkdir -p /content/data/WESAD
!cp -r /root/.cache/kagglehub/datasets/orvile/wesad-wearable-stress-affect-detection-dataset/versions/1/WESAD/ /content/data/
!cp -r /kaggle/input/wesad-wearable-stress-affect-detection-dataset/WESAD/ /content/data/

In [ ]:
!python data_wrangling.py

In [ ]:
from readme_parser import rparser
rparser()

In [ ]:
# Train a tree-based model on the merged WESAD feature table and rank features by importance
!python tree_feature_importance.py \
--data data/m14_merged.csv \
--output-dir artifacts \
--n-splits 5 --n-estimators 500

In [ ]:
# !python tree_feature.py \
#   --data data/m14_merged.csv \
#   --output-dir artifacts2 \
#   --n-splits 5 \
#   --n-estimators 500 \
#   --drop-cols

In [ ]:
# Train RandomForest and XGBoost after removing the requested features
# This loads the wrangled CSV, drops the columns, runs 5-fold Stratified CV, and prints accuracy and F1-macro.

#%cd /content/WESAD

import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# 1) Load data
df = pd.read_csv("data/m14_merged.csv")

# 2) Columns to drop
to_remove = ['EDA_tonic_min', 'ACC_x_min', 'ACC_y_min', 'ACC_z_min', 'TEMP_min', 'BVP_min']
label_col = 'label'  # change if your label column is different

# 3) Prepare X, y
drop_cols = [c for c in to_remove if c in df.columns]
X = df.drop(columns=[label_col] + drop_cols, errors='ignore')
y = df[label_col].astype(int)

print("Dropped columns:", drop_cols)
print("X shape:", X.shape)

# 4) 5-fold Stratified CV
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

rf_accs, rf_f1s = [], []
xgb_accs, xgb_f1s = [], []

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, y), 1):
    X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
    y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

    # RandomForest
    rf = RandomForestClassifier(
        n_estimators=500,
        random_state=42,
        n_jobs=-1
    )
    rf.fit(X_tr, y_tr)
    rf_pred = rf.predict(X_va)
    rf_accs.append(accuracy_score(y_va, rf_pred))
    rf_f1s.append(f1_score(y_va, rf_pred, average='macro'))

    # XGBoost
    xgb = XGBClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1,
        tree_method='hist',
        eval_metric='logloss'
    )
    xgb.fit(X_tr, y_tr)
    xgb_pred = xgb.predict(X_va)
    xgb_accs.append(accuracy_score(y_va, xgb_pred))
    xgb_f1s.append(f1_score(y_va, xgb_pred, average='macro'))

print(f"RandomForest — Accuracy: {np.mean(rf_accs):.4f} ± {np.std(rf_accs):.4f} | F1-macro: {np.mean(rf_f1s):.4f} ± {np.std(rf_f1s):.4f}")
print(f"XGBoost     — Accuracy: {np.mean(xgb_accs):.4f} ± {np.std(xgb_accs):.4f} | F1-macro: {np.mean(xgb_f1s):.4f} ± {np.std(xgb_f1s):.4f}")


# Save the reduced-feature results to artifacts
import json, os
os.makedirs("artifacts", exist_ok=True)
results = {
    "dropped_columns": drop_cols,
    "rf_accuracy_mean": float(np.mean(rf_accs)),
    "rf_accuracy_std": float(np.std(rf_accs)),
    "rf_f1_macro_mean": float(np.mean(rf_f1s)),
    "rf_f1_macro_std": float(np.std(rf_f1s)),
    "xgb_accuracy_mean": float(np.mean(xgb_accs)),
    "xgb_accuracy_std": float(np.std(xgb_accs)),
    "xgb_f1_macro_mean": float(np.mean(xgb_f1s)),
    "xgb_f1_macro_std": float(np.std(xgb_f1s)),
}
with open("artifacts/reduced_features_results.json", "w") as f:
    json.dump(results, f, indent=2)
print("Saved to artifacts/reduced_features_results.json")